# Notebook on the Douglas Prices.

### Nothing big happens, just some rearrangements, data cleaning etc

In [35]:
# import os
# import json
# import pandas as pd

# # Directory containing the JSON files
# json_dir = "idealo_prices/queries/brands_prices"

# # List to store parsed data
# data = []

# # Iterate over each JSON file
# for file in os.listdir(json_dir):
#     if file.endswith(".json"):
#         brand = file.replace(".json", "")  # Extract brand name from filename
#         with open(os.path.join(json_dir, file), "r", encoding="utf-8") as f:
#             perfumes = json.load(f)  # Load JSON data
        
#         # Extract data from each perfume entry
#         for perfume in perfumes:
#             row = {
#                 "brand": brand,
#                 "perfume_name": perfume["perfume_name"],
#                 "url": perfume["url"]
#             }
            
#             # Extract price information for each bottle size
#             for size, price_info in perfume["prices"].items():
#                 row[f"{size} original_price"] = price_info.get("original_price")
#                 row[f"{size} reduced_price"] = price_info.get("reduced_price")
            
#             data.append(row)

# # Convert to DataFrame
# df = pd.DataFrame(data)

# # Fill NaN values with appropriate defaults (optional)
# # df.fillna("", inplace=True)

# # Display DataFrame
# print(df)

In [36]:
# df.to_csv('data/douglas_prices_df.csv')

In [37]:
import pandas as pd

In [38]:
df_douglas = pd.read_csv('data/douglas_prices_df.csv')

In [39]:
from rapidfuzz import process, fuzz

# Target string
target_perfume = "Remix Cologne Anniversary Edition"

# Get the best match from df_douglas['perfume_name']
best_match, score, _ = process.extractOne(target_perfume, df_douglas['perfume_name'], scorer=fuzz.ratio)

print(f"Best match: {best_match} (Score: {score})")

Best match: Abercrombie & Fitch Eau de Cologne Fierce Pride Edition (Score: 54.54545454545454)


In [40]:
df_douglas[(df_douglas['brand'] == '4711') & (df_douglas['perfume_name'] == 'Remix Cologne Anniversary Edition')]

,Unnamed: 0,brand,perfume_name,url,100 ml original_price,100 ml reduced_price,30 ml original_price,30 ml reduced_price,50 ml original_price,50 ml reduced_price,125 ml original_price,125 ml reduced_price,150 ml original_price,150 ml reduced_price,40 ml original_price,40 ml reduced_price,75 ml original_price,75 ml reduced_price,60 ml original_price,60 ml reduced_price,110 ml original_price,110 ml reduced_price,90 ml original_price,90 ml reduced_price,200 ml original_price,200 ml reduced_price,80 ml original_price,80 ml reduced_price,250 ml original_price,250 ml reduced_price,750 ml original_price,750 ml reduced_price,600 ml original_price,600 ml reduced_price,65 ml original_price,65 ml reduced_price,120 ml original_price,120 ml reduced_price,10 ml original_price,10 ml reduced_price,36 ml original_price,36 ml reduced_price,25 ml original_price,25 ml reduced_price,180 ml original_price,180 ml reduced_price,20 ml original_price,20 ml reduced_price,300 ml original_price,300 ml reduced_price,170 ml original_price,170 ml reduced_price,8 ml original_price,8 ml reduced_price,15 ml original_price,15 ml reduced_price,35 ml original_price,35 ml reduced_price,177 ml original_price,177 ml reduced_price,27 ml original_price,27 ml reduced_price,225 ml original_price,225 ml reduced_price,95 ml original_price,95 ml reduced_price,53 ml original_price,53 ml reduced_price,45 ml original_price,45 ml reduced_price,14 ml original_price,14 ml reduced_price,118 ml original_price,118 ml reduced_price,236 ml original_price,236 ml reduced_price,3 ml original_price,3 ml reduced_price,12 ml original_price,12 ml reduced_price,5 ml original_price,5 ml reduced_price,85 ml original_price,85 ml reduced_price,175 ml original_price,175 ml reduced_price,21 ml original_price,21 ml reduced_price,140 ml original_price,140 ml reduced_price,500 ml original_price,500 ml reduced_price,97 ml original_price,97 ml reduced_price,290 ml original_price,290 ml reduced_price,70 ml original_price,70 ml reduced_price,11 ml original_price,11 ml reduced_price,33 ml original_price,33 ml reduced_price,350 ml original_price,350 ml reduced_price,400 ml original_price,400 ml reduced_price,160 ml original_price,160 ml reduced_price,240 ml original_price,240 ml reduced_price,59 ml original_price,59 ml reduced_price,26 ml original_price,26 ml reduced_price,88 ml original_price,88 ml reduced_price,650 ml original_price,650 ml reduced_price,105 ml original_price,105 ml reduced_price,18 ml original_price,18 ml reduced_price,87 ml original_price,87 ml reduced_price,194 ml original_price,194 ml reduced_price,29 ml original_price,29 ml reduced_price,avg_original_price_per_liter,avg_reduced_price_per_liter


In [41]:
og_df = pd.read_pickle('encoded_perfumes.pkl')

In [51]:
og_df['complete_name'] = og_df['brand'] + ' ' + og_df['name']

In [52]:
og_df['complete_name']

0             4711 Remix Cologne Anniversary Edition
1                       Azzaro Solarissimo - Levanzo
2                            Mugler A*Men Ultra Zest
3                   Acqua di Parma Acqua Nobile Rosa
4                   Fragrance Du Bois Minuit et Demi
                            ...                     
14503                              Mazzolari Hartley
14504                         Adi ale Van Gethsemane
14505    Bentley Bentley Infinite Rush White Edition
14506                     Bohoboco Olibanum Gardenia
14507                    MCM Obelisk MCM 1985 Parfum
Name: complete_name, Length: 14508, dtype: object

In [53]:
df_douglas['complete_name'] = df_douglas['brand'] + ' ' + df_douglas['perfume_name']

In [63]:
import pandas as pd
from rapidfuzz import process, fuzz

# Example DataFrames
df1 = og_df  # First DataFrame
df2 = df_douglas  # Second DataFrame

# Get unique values from the 'complete_name' column in df2 to match against
choices = df1['complete_name'].unique()

# Function to perform fuzzy matching
def fuzzy_match_name(name, choices):
    if pd.isna(name):  # Handle NaN values
        return None
    match, score, _ = process.extractOne(name, choices, scorer=fuzz.ratio)  # Get best match
    return match if score > 4 else None  # Only return matches with similarity > 80%

# Apply fuzzy matching on df1
df2['matched_name'] = df2['complete_name'].apply(lambda x: fuzzy_match_name(x, choices))

# Merge df1 with df2 using the fuzzy-matched name
merged_df = df2.merge(df1, left_on='complete_name', right_on='matched_name', suffixes=('_df1', '_df2'))

# Drop the helper column if no longer needed
# merged_df.drop(columns=['matched_name'], inplace=True)

# Display result
print(merged_df)

       Unnamed: 0 brand_df1                                                perfume_name                                                 url_df1  100 ml original_price  100 ml reduced_price  30 ml original_price  30 ml reduced_price  50 ml original_price  50 ml reduced_price  125 ml original_price  125 ml reduced_price  150 ml original_price  150 ml reduced_price  40 ml original_price  40 ml reduced_price  75 ml original_price  75 ml reduced_price  60 ml original_price  60 ml reduced_price  110 ml original_price  110 ml reduced_price  90 ml original_price  90 ml reduced_price  200 ml original_price  200 ml reduced_price  80 ml original_price  80 ml reduced_price  250 ml original_price  250 ml reduced_price  750 ml original_price  750 ml reduced_price  600 ml original_price  600 ml reduced_price  65 ml original_price  65 ml reduced_price  120 ml original_price  120 ml reduced_price  10 ml original_price  10 ml reduced_price  36 ml original_price  36 ml reduced_price  25 ml original_price

In [66]:
df1[(df1['brand']=='XerJoff') & (df1['collection']=='1861')]

name    brand    year collection flakon_designer perfumer                                                 top_notes                                middle_notes                               base_notes                                                                                                                               all_notes                                                 url  Duft.rating  Duft.number_of_ratings  Haltbarkeit.rating  Haltbarkeit.number_of_ratings  Sillage.rating  Sillage.number_of_ratings  Flakon.rating  Flakon.number_of_ratings  Preis-Leistungs-Verhältnis.rating  Preis-Leistungs-Verhältnis.number_of_ratings  scent.0  scent.1  scent.2  scent.3  scent.4  scent.5  scent.6  scent.7  scent.8  scent.9  scent.10  durability.0  durability.1  durability.2  durability.3  durability.4  durability.5  durability.6  durability.7  durability.8  durability.9  durability.10  sillage.0  sillage.1  sillage.2  sillage.3  sillage.4  sillage.5  sillage.6  sillage.7  sillage.8  sillage.9  sillage.10  bottle.0  bottle.1  bottle.2  bottle.3  bottle.4  bottle.5  bottle.6  bottle.7  bottle.8  bottle.9  bottle.10  pricing.0  pricing.1  pricing.2  pricing.3  pricing.4  pricing.5  pricing.6  pricing.7  pricing.8  pricing.9  pricing.10  Zitrus  Würzig   Grün  Fruchtig  Frisch  Blumig  Herren  Damen  Klassisch  Modern  Winter  Herbst  Sommer  Frühling  Täglich  Sport  Freizeit  Ausgehen  Arbeit  Abend  pricing  Synthetisch    Süß  Holzig  Aquatisch  Gourmand  Pudrig  Orientalisch  Cremig  Ledrig  Harzig  Rauchig  Erdig  Animalisch  Fougère  Chypre  durability  sillage  bottle  scent  2-Methylundecanal  Abelmoschus  Abend-Levkoje  Abrialis-Lavendel  Absinth  Acajouholz  Acerolakirsche  Ackerwinde  Adesmia boronioides  Adlerholzharz  Adrenalin  Affenbrotbaum  Afrikanischer Rosmarin  Afrikanisches Grenadill  Agave  Agavensirup  Ahorn  Ahornholz  Ahornsirup  Ajowan  Akanthus  Akazie  Akazienholz  Akazienhonig  Akaziensamen  Akigalawood®  Alba-Rose  Alba-Rose Attar  Alba-Rose Otto  Aldehyde  Aldron®  Aleppo-Kiefer  Alge Absolue  Algen  Almdudler  Aloe-Vera  Alpenveilchen  Alpenveilchenblatt  Alpenzeder  Alpha-Terpineol  Alphonso-Mango  Alraune  Amalfi-Bergamotte  Amalfi-Limette  Amalfi-Orange  Amalfi-Orangenblüte  Amalfi-Traube  Amalfizitrone  Amaranth-Holz  Amarenakirsche  Amaretto  Amaro Siciliano  Amaryllis  Amazonas-Riesenseerose  Amazonaslilie  Amber  Amber Absolue  Amber Core  Amber Xtreme™  Amberholz  Amberketal  Ambermax®  Ambermor  Ambernoten  Ambertonic®  Ambra  Ambra Absolue  Ambramone™  Ambranum  Ambrarome  Ambrarome Absolue  Ambre 84-Akkord  Ambrein  Ambreinol  Ambrette  Ambrettesamen  Ambrettesamen Absolue  Ambrettolid  Ambrocenide®  Ambrofix  Ambrostar  Ambrox  Ambrox Super  Ambroxan  Amburana cearensis  Amerikanische Wildpflaume  Amyris  Ananas  Ananas Orpur®  Ananas-Sorbet  Ananasblatt  Ananasblüte  Ananassaft  Angelika  Angelikablüte  Angelikasamen  Angelikawurzel  Anis  Anjoubirne  Annurca-Apfel  Apfel  Apfel-Martini  Apfel-Sorbet  Apfel-Whiskey  Apfelblatt  Apfelblüte  Apfelbranntwein  Apfelkuchen  Apfelmus  Aprikose  Aprikosen-Likör  Aprikosen-Sorbet  Aprikosenblatt  Aprikosenblüte  Aprikosenholz  Aprikosennektar  Aprikosenschale  Aquaflora  Aqual™  Aquozone  Arabica-Kaffee  Arabica-Kaffee Absolue  Arctical™  Arganie  Arizona-Wacholder  Arnika  Arnika Absolue  Aronstabblüte  Artemisia  Asant  Asche  Asphalt  Assam-Oud  Atlaszeder  Atlaszeder Absolue  Atlaszeder Orpur®  Aurelione®  Austernpflanze  Australisches Sandelholz  Australisches Sandelholz Orpur®  Avocado  Azalee  Açaí  Baccara-Rose  Backpflaume  Baiser  Bakhoor  Bakul Attar  Baldrian  Balkan-Eichenmoos  Balkan-Immortelle Absolue  Balkan-Storchschnabel  Balkan-Tabak  Balkan-Tabak Absolue  Balkan-Tabakblüte  Balkan-Wacholderbeere  Balsaholz  Balsam  Balsambaum  Balsamtanne  Balsamtanne Absolue  Balsamtannenharz  Bambus  Bambusblatt  Bambusblüte  Bambussaft  Banane  Bananenblatt  Bananenblüte  Bananenbrot  Basilikum  Basilikum Absolue  Basilikum-Sorbet  Basilik

In [69]:
df2[(df2['brand'] == 'XERJOFF') & df2['perfume_name'].str.contains('1861', na=False)]

,Unnamed: 0,brand,perfume_name,url,100 ml original_price,100 ml reduced_price,30 ml original_price,30 ml reduced_price,50 ml original_price,50 ml reduced_price,125 ml original_price,125 ml reduced_price,150 ml original_price,150 ml reduced_price,40 ml original_price,40 ml reduced_price,75 ml original_price,75 ml reduced_price,60 ml original_price,60 ml reduced_price,110 ml original_price,110 ml reduced_price,90 ml original_price,90 ml reduced_price,200 ml original_price,200 ml reduced_price,80 ml original_price,80 ml reduced_price,250 ml original_price,250 ml reduced_price,750 ml original_price,750 ml reduced_price,600 ml original_price,600 ml reduced_price,65 ml original_price,65 ml reduced_price,120 ml original_price,120 ml reduced_price,10 ml original_price,10 ml reduced_price,36 ml original_price,36 ml reduced_price,25 ml original_price,25 ml reduced_price,180 ml original_price,180 ml reduced_price,20 ml original_price,20 ml reduced_price,300 ml original_price,300 ml reduced_price,170 ml original_price,170 ml reduced_price,8 ml original_price,8 ml reduced_price,15 ml original_price,15 ml reduced_price,35 ml original_price,35 ml reduced_price,177 ml original_price,177 ml reduced_price,27 ml original_price,27 ml reduced_price,225 ml original_price,225 ml reduced_price,95 ml original_price,95 ml reduced_price,53 ml original_price,53 ml reduced_price,45 ml original_price,45 ml reduced_price,14 ml original_price,14 ml reduced_price,118 ml original_price,118 ml reduced_price,236 ml original_price,236 ml reduced_price,3 ml original_price,3 ml reduced_price,12 ml original_price,12 ml reduced_price,5 ml original_price,5 ml reduced_price,85 ml original_price,85 ml reduced_price,175 ml original_price,175 ml reduced_price,21 ml original_price,21 ml reduced_price,140 ml original_price,140 ml reduced_price,500 ml original_price,500 ml reduced_price,97 ml original_price,97 ml reduced_price,290 ml original_price,290 ml reduced_price,70 ml original_price,70 ml reduced_price,11 ml original_price,11 ml reduced_price,33 ml original_price,33 ml reduced_price,350 ml original_price,350 ml reduced_price,400 ml original_price,400 ml reduced_price,160 ml original_price,160 ml reduced_price,240 ml original_price,240 ml reduced_price,59 ml original_price,59 ml reduced_price,26 ml original_price,26 ml reduced_price,88 ml original_price,88 ml reduced_price,650 ml original_price,650 ml reduced_price,105 ml original_price,105 ml reduced_price,18 ml original_price,18 ml reduced_price,87 ml original_price,87 ml reduced_price,194 ml original_price,194 ml reduced_price,29 ml original_price,29 ml reduced_price,avg_original_price_per_liter,avg_reduced_price_per_liter,is_in_set,complete_name,matched_name
1374,1374,XERJOFF,XERJOFF Parfum XJ 1861 DECAS,https://www.douglas.de/de/p/5010934022,245.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2450.0,NaN,False,XERJOFF XERJOFF Parfum XJ 1861 DECAS,Frau Tonis Parfum № 11 Si Tu Savais
1421,1421,XERJOFF,XERJOFF Eau de Parfum XJ 1861 Renaissance,https://www.douglas.de/de/p/5009877043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fals

In [70]:
df2[['brand', 'perfume_name']]

,brand,perfume_name
0,Berdoues,Berdoues Eau de Toilette ROSE & LITCHI
1,Berdoues,Berdoues Eau de Toilette MONOI & COCO
2,Berdoues,Berdoues Eau de Toilette THÉ VERT & FREE
3,Berdoues,Berdoues Eau de Parfum Verveine & Clémentine
4,Berdoues,Berdoues Eau de Parfum Collection Grand Crus Guaria Morada
...,...,...
5882,Creed,Creed Eau de Parfum Millesime for Women Love In White For Summer
5883,Creed,Creed Eau de Parfum Millesime for Men Original Santal
5884,Creed,Creed Parfum Millesime for Women Wind Flowers
5885,Creed,Creed Eau de Parfum Acqua Originale Citrus Bigarade


In [73]:
import  re

# Ensure columns are strings and fill NaN values
df2['brand'] = df2['brand'].fillna("").astype(str)
df2['perfume_name'] = df2['perfume_name'].fillna("").astype(str)

# Extract perfume type
perfume_type_pattern = r"(Eau de Parfum|Eau de Toilette|Parfum|Eau de Cologne|Eau Fraîche|Parfum)"
df2['perfume_type'] = df2['perfume_name'].str.extract(perfume_type_pattern, expand=False)

# Strip brand name from perfume_name (if present at the start)
df2['perfume_name_cleaned'] = df2.apply(
    lambda row: re.sub(f"^{re.escape(row['brand'])}\\s*", "", row['perfume_name']), axis=1
)

# Display result
print(df2.head())

   Unnamed: 0     brand                                                perfume_name                                     url  100 ml original_price  100 ml reduced_price  30 ml original_price  30 ml reduced_price  50 ml original_price  50 ml reduced_price  125 ml original_price  125 ml reduced_price  150 ml original_price  150 ml reduced_price  40 ml original_price  40 ml reduced_price  75 ml original_price  75 ml reduced_price  60 ml original_price  60 ml reduced_price  110 ml original_price  110 ml reduced_price  90 ml original_price  90 ml reduced_price  200 ml original_price  200 ml reduced_price  80 ml original_price  80 ml reduced_price  250 ml original_price  250 ml reduced_price  750 ml original_price  750 ml reduced_price  600 ml original_price  600 ml reduced_price  65 ml original_price  65 ml reduced_price  120 ml original_price  120 ml reduced_price  10 ml original_price  10 ml reduced_price  36 ml original_price  36 ml reduced_price  25 ml original_price  25 ml reduced_pric

In [103]:
# Ensure columns are strings and handle NaN values
df2['brand'] = df2['brand'].fillna("").astype(str)
df2['perfume_name'] = df2['perfume_name'].fillna("").astype(str)

# Define pattern for perfume type
perfume_type_pattern = r"(Eau de Parfum|Eau de Toilette|Parfum|Eau de Cologne|Eau Fraîche|Parfum|Extrait de Parfum|Eau Fraiche)"

# Extract perfume type into a new column
df2['perfume_type'] = df2['perfume_name'].str.extract(perfume_type_pattern, expand=False)

# Clean perfume name: remove brand name and perfume type
def clean_perfume_name(row):
    name = row['perfume_name']
    
    # Remove brand name if present at the beginning
    name = re.sub(f"^{re.escape(row['brand'])}\\s*", "", name)
    
    # Remove perfume type
    name = re.sub(perfume_type_pattern, "", name).strip()
    
    return name

df2['perfume_name_cleaned'] = df2.apply(clean_perfume_name, axis=1)

# Display result
print(df2[['brand', 'perfume_name', 'perfume_type', 'perfume_name_cleaned']].head())

      brand                                                perfume_name     perfume_type                 perfume_name_cleaned
0  Berdoues                      Berdoues Eau de Toilette ROSE & LITCHI  Eau de Toilette                        ROSE & LITCHI
1  Berdoues                       Berdoues Eau de Toilette MONOI & COCO  Eau de Toilette                         MONOI & COCO
2  Berdoues                    Berdoues Eau de Toilette THÉ VERT & FREE  Eau de Toilette                      THÉ VERT & FREE
3  Berdoues                Berdoues Eau de Parfum Verveine & Clémentine    Eau de Parfum                Verveine & Clémentine
4  Berdoues  Berdoues Eau de Parfum Collection Grand Crus Guaria Morada    Eau de Parfum  Collection Grand Crus Guaria Morada


In [104]:
test_df = df2[['brand', 'perfume_name', 'perfume_type', 'perfume_name_cleaned', 'avg_original_price_per_liter', 'avg_reduced_price_per_liter']]

In [105]:
test_df[['avg_original_price_per_liter', 'avg_reduced_price_per_liter']] = test_df[['avg_original_price_per_liter', 'avg_reduced_price_per_liter']].round(2)

/var/folders/lq/blpghcxd0b9cm2tsrvlqq9g40000gn/T/ipykernel_19426/3279692257.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[['avg_original_price_per_liter', 'avg_reduced_price_per_liter']] = test_df[['avg_original_price_per_liter', 'avg_reduced_price_per_liter']].round(2)


In [108]:
test_df.dropna(subset=['perfume_type'], inplace=True)
test_df.drop(test_df[test_df['perfume_name_cleaned'].str.contains('Duftset', na=False)].index, inplace=True)
test_df.dropna(subset=['avg_original_price_per_liter'], inplace=True)

/var/folders/lq/blpghcxd0b9cm2tsrvlqq9g40000gn/T/ipykernel_19426/2250217278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.dropna(subset=['perfume_type'], inplace=True)
/var/folders/lq/blpghcxd0b9cm2tsrvlqq9g40000gn/T/ipykernel_19426/2250217278.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(test_df[test_df['perfume_name_cleaned'].str.contains('Duftset', na=False)].index, inplace=True)
/var/folders/lq/blpghcxd0b9cm2tsrvlqq9g40000gn/T/ipykernel_19426/2250217278.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat